In [3]:
import pandas as pd
import math
import numpy as np

In [2]:
# originally used a cropped version of the data because my computer couldn't open it all
#d = pd.read_csv("EMS_Incident_Dispatch_Data_cropped.csv")

# Varun was able to parse to get 2018 data (which was our original plan)
d = pd.read_csv("2018.csv")

## Determining which unit of area is smallest

###### counted number of different areas for each location category to determine which we should use

In [13]:
ida_distinct = []
for area in d['INCIDENT_DISPATCH_AREA']:
    if area not in ida_distinct:
        ida_distinct.append(area)
len(ida_distinct) #34

comDist_distinct = []
for area in d['COMMUNITYDISTRICT']:
    if not math.isnan(area) and area not in comDist_distinct:
        comDist_distinct.append(area)
len(comDist_distinct) #71

zip_distinct = []
for area in d['ZIPCODE']:
    if not math.isnan(area) and area not in zip_distinct:
        zip_distinct.append(area)
len(zip_distinct) #236

cc_distinct = []
for area in d['CITYCOUNCILDISTRICT']:
    if not math.isnan(area) and area not in cc_distinct:
        cc_distinct.append(area)
len(cc_distinct) #51

comDist_distinct = []
for area in d['COMMUNITYDISTRICT']:
    if not math.isnan(area) and area not in comDist_distinct:
        comDist_distinct.append(area)
len(comDist_distinct) #71

# CONCLUSION: use ZIPCODE

51

###### mapping community districts to incident dispatch areas to try to determine how calls are routed based on location

In [ ]:
combo_distinct = {}
t = d[['INCIDENT_DISPATCH_AREA','COMMUNITYDISTRICT']]

for index, row in t.iterrows(): 
    if not math.isnan(row['COMMUNITYDISTRICT']):
        if row['INCIDENT_DISPATCH_AREA'] not in combo_distinct.keys():
            combo_distinct[row['INCIDENT_DISPATCH_AREA']] = [row['COMMUNITYDISTRICT']]
        else:
            if row['COMMUNITYDISTRICT'] not in combo_distinct[row['INCIDENT_DISPATCH_AREA']]:
                combo_distinct[row['INCIDENT_DISPATCH_AREA']].append(row['COMMUNITYDISTRICT'])
combo_distinct

In [ ]:
rev_combo_distinct = {}

for index, row in t.iterrows(): 
    if not math.isnan(row['COMMUNITYDISTRICT']):
        if row['COMMUNITYDISTRICT'] not in rev_combo_distinct.keys():
            rev_combo_distinct[row['COMMUNITYDISTRICT']] = [row['INCIDENT_DISPATCH_AREA']]
        else:
            if row['INCIDENT_DISPATCH_AREA'] not in rev_combo_distinct[row['COMMUNITYDISTRICT']]:
                rev_combo_distinct[row['COMMUNITYDISTRICT']].append(row['INCIDENT_DISPATCH_AREA'])
rev_combo_distinct

In [ ]:
num_per_comDist = []
for x in rev_combo_distinct.keys():
    num_per_comDist.append(len(rev_combo_distinct[x]))
np.mean(num_per_comDist)
np.std(num_per_comDist) #4.312

#CONCLUSION: for community districts, we could not find a correlation

###### tried mapping to boroughs to see if they calls divided cleanly there

In [29]:
borough_combo_distinct = {}
t = d[['INCIDENT_DISPATCH_AREA','BOROUGH']]

for index, row in t.iterrows(): 
    if row['INCIDENT_DISPATCH_AREA'] not in borough_combo_distinct.keys():
        borough_combo_distinct[row['INCIDENT_DISPATCH_AREA']] = [row['BOROUGH']]
    else:
        if row['BOROUGH'] not in borough_combo_distinct[row['INCIDENT_DISPATCH_AREA']]:
            borough_combo_distinct[row['INCIDENT_DISPATCH_AREA']].append(row['BOROUGH'])
borough_combo_distinct

{'B3': ['BRONX',
  'RICHMOND / STATEN ISLAND',
  'QUEENS',
  'MANHATTAN',
  'BROOKLYN'],
 'B1': ['BRONX',
  'MANHATTAN',
  'BROOKLYN',
  'QUEENS',
  'RICHMOND / STATEN ISLAND'],
 'Q3': ['QUEENS', 'BROOKLYN', 'MANHATTAN'],
 'B2': ['BRONX', 'MANHATTAN', 'BROOKLYN', 'QUEENS'],
 'M3': ['MANHATTAN', 'BROOKLYN', 'QUEENS', 'BRONX'],
 'S2': ['RICHMOND / STATEN ISLAND',
  'MANHATTAN',
  'BROOKLYN',
  'BRONX',
  'QUEENS',
  'UNKNOWN'],
 'S1': ['RICHMOND / STATEN ISLAND',
  'MANHATTAN',
  'BROOKLYN',
  'BRONX',
  'QUEENS'],
 'M1': ['MANHATTAN',
  'BROOKLYN',
  'RICHMOND / STATEN ISLAND',
  'BRONX',
  'QUEENS',
  'UNKNOWN'],
 'B4': ['BRONX', 'MANHATTAN', 'BROOKLYN'],
 'X2': ['MANHATTAN', 'BROOKLYN', 'QUEENS', 'RICHMOND / STATEN ISLAND'],
 'M2': ['MANHATTAN', 'BROOKLYN', 'BRONX', 'QUEENS'],
 'K3': ['BROOKLYN',
  'MANHATTAN',
  'BRONX',
  'QUEENS',
  'RICHMOND / STATEN ISLAND',
  'UNKNOWN'],
 'K5': ['BROOKLYN',
  'BRONX',
  'MANHATTAN',
  'RICHMOND / STATEN ISLAND',
  'QUEENS'],
 'K4': ['BROOKLYN',


In [31]:
boroughs_per_disp = []
for x in borough_combo_distinct.keys():
    boroughs_per_disp.append(len(borough_combo_distinct[x]))
np.std(boroughs_per_disp)

#CONCLUSION: dispatch stations served outside of their borough frequently

0.9471318934179187

### We then determined this system would be easier to model if we only looked at one borough, not the entire city. This way we could have more historical data to work with without exceeding reasonable computation times 
##### to aid in this decision, we wanted to determine which city had the most calls

In [4]:
borough_dict = {}
for index, row in d.iterrows(): 
    if row['BOROUGH'] not in borough_dict.keys():
        borough_dict[row['BOROUGH']] = 1
    else:
        borough_dict[row['BOROUGH']] += 1
borough_dict

{'BROOKLYN': 428457,
 'QUEENS': 293356,
 'MANHATTAN': 390801,
 'BRONX': 358862,
 'RICHMOND / STATEN ISLAND': 63139,
 'UNKNOWN': 8}

In [1]:
total = 428457 + 293356 + 390801 + 358862 + 63139 + 8
brooklyn = 428457/total
queens = 293356/total
manhat = 390801/total
bronx = 358862/total
rsi = 63139/total

print("brkl: " + str(brooklyn) + ", qu: " + str(queens) + ", manht: " + str(manhat) + ", bronx: " + str(bronx) + ", rsi: " + str(rsi))

#CONCLUSION: All the boroughs have similar proportions. W
#      we will use Bronx because it has the most calls per number of stations

brkl: 0.2791936521217263, qu: 0.19115834964026995, manht: 0.25465602952647004, bronx: 0.23384375185306097, rsi: 0.04114300385176033


#### Now we want to look at what attributes we will need from the dataset for our projects 

In [5]:
d

,Unnamed: 0,CAD_INCIDENT_ID,INCIDENT_DATETIME,INITIAL_CALL_TYPE,INITIAL_SEVERITY_LEVEL_CODE,FINAL_CALL_TYPE,FINAL_SEVERITY_LEVEL_CODE,FIRST_ASSIGNMENT_DATETIME,VALID_DISPATCH_RSPNS_TIME_INDC,DISPATCH_RESPONSE_SECONDS_QY,...,ZIPCODE,POLICEPRECINCT,CITYCOUNCILDISTRICT,COMMUNITYDISTRICT,COMMUNITYSCHOOLDISTRICT,CONGRESSIONALDISTRICT,REOPEN_INDICATOR,SPECIAL_EVENT_INDICATOR,STANDBY_INDICATOR,TRANSFER_INDICATOR
0,0,183654386,2018-12-31T23:59:46.000,CARDBR,2,CARDBR,2,2018-12-31T23:59:54.000,Y,8,...,11201.0,84.0,35.0,302.0,13.0,8.0,N,N,N,N
1,1,183654385,2018-12-31T23:59:20.000,MVAINJ,4,MVAINJ,4,2019-01-01T00:01:10.000,Y,110,...,11416.0,102.0,32.0,409.0,27.0,7.0,N,N,N,N
2,2,183654384,2018-12-31T23:58:45.000,SICK,6,SICK,6,2019-01-01T00:00:43.000,Y,118,...,11369.0,115.0,21.0,480.0,30.0,14.0,N,N,N,N
3,3,183654383,2018-12-31T23:58:14.000,INJMAJ,3,INJMAJ,3,2018-12-31T23:58:22.000,Y,8,...,11214.0,62.0,47.0,311.0,20.0,11.0,N,N,N,N
4,4,183654382,2018-12-31T23:57:42.000,UNC,2,UNC,2,2019-01-01T00:00:39.000,Y,177,...,10019.0,18.0,4.0,105.0,2.0,12.0,N,N,N,N
5,5,183654381,2018-12-31T23:57:41.000,SICMIN,7,SICMIN,7,2018-12-31T23:58:01.000,Y,20,...,11208.0,75.0,42.0,305.0,19.0,8.0,N,N,N,N
6,6,183654380,2018-12-31T23:57:06.000,UNKNOW,4,UNKNOW,4,2018-12-31T23:57:17.000,Y,11,...,11435.0,107.0,24.0,408.0,28.0,6.0,N,N,N,N
7,7,183654379,2018-12-31T23:57:04.000,CARDBR,2,CARDBR,2,2018-12-31T23:57:16.000,Y,12,...,10465.0,45.0,13.0,210.0,8.0,14.0,N,N,N,N
8,8,183654377,2018-12-31T23:55:49.000,INJURY,5,INJURY,5,2018-12-31T23:56:20.000,Y,31,...,11691.0,101.0,31.0,414.0,27.0,5.0,N,N,N,N
9,9,183654376,2018-12-31T23:55:27.000,INJURY,5,INJURY,5,2018-12-31T23:56:02.000,Y,35,...,10453.0,46.0,14.0,205.0,9.0,15.0,N,N,N,N


In [6]:
d.columns

Index(['Unnamed: 0', 'CAD_INCIDENT_ID', 'INCIDENT_DATETIME',
       'INITIAL_CALL_TYPE', 'INITIAL_SEVERITY_LEVEL_CODE', 'FINAL_CALL_TYPE',
       'FINAL_SEVERITY_LEVEL_CODE', 'FIRST_ASSIGNMENT_DATETIME',
       'VALID_DISPATCH_RSPNS_TIME_INDC', 'DISPATCH_RESPONSE_SECONDS_QY',
       'FIRST_ACTIVATION_DATETIME', 'FIRST_ON_SCENE_DATETIME',
       'VALID_INCIDENT_RSPNS_TIME_INDC', 'INCIDENT_RESPONSE_SECONDS_QY',
       'INCIDENT_TRAVEL_TM_SECONDS_QY', 'FIRST_TO_HOSP_DATETIME',
       'FIRST_HOSP_ARRIVAL_DATETIME', 'INCIDENT_CLOSE_DATETIME',
       'HELD_INDICATOR', 'INCIDENT_DISPOSITION_CODE', 'BOROUGH',
       'INCIDENT_DISPATCH_AREA', 'ZIPCODE', 'POLICEPRECINCT',
       'CITYCOUNCILDISTRICT', 'COMMUNITYDISTRICT', 'COMMUNITYSCHOOLDISTRICT',
       'CONGRESSIONALDISTRICT', 'REOPEN_INDICATOR', 'SPECIAL_EVENT_INDICATOR',
       'STANDBY_INDICATOR', 'TRANSFER_INDICATOR'],
      dtype='object')

In [8]:
d2 = d[['INCIDENT_DATETIME', 'INITIAL_CALL_TYPE', 'INITIAL_SEVERITY_LEVEL_CODE', 'FINAL_CALL_TYPE', 'FINAL_SEVERITY_LEVEL_CODE', 'FIRST_ASSIGNMENT_DATETIME', 'FIRST_ON_SCENE_DATETIME', 'BOROUGH', 'INCIDENT_DISPATCH_AREA', 'ZIPCODE']]

In [10]:
d2

,INCIDENT_DATETIME,INITIAL_CALL_TYPE,INITIAL_SEVERITY_LEVEL_CODE,FINAL_CALL_TYPE,FINAL_SEVERITY_LEVEL_CODE,FIRST_ASSIGNMENT_DATETIME,FIRST_ON_SCENE_DATETIME,BOROUGH,INCIDENT_DISPATCH_AREA,ZIPCODE
0,2018-12-31T23:59:46.000,CARDBR,2,CARDBR,2,2018-12-31T23:59:54.000,2019-01-01T00:07:27.000,BROOKLYN,K6,11201.0
1,2018-12-31T23:59:20.000,MVAINJ,4,MVAINJ,4,2019-01-01T00:01:10.000,2019-01-01T00:07:19.000,QUEENS,Q3,11416.0
2,2018-12-31T23:58:45.000,SICK,6,SICK,6,2019-01-01T00:00:43.000,2019-01-01T00:14:18.000,QUEENS,Q6,11369.0
3,2018-12-31T23:58:14.000,INJMAJ,3,INJMAJ,3,2018-12-31T23:58:22.000,2019-01-01T00:04:10.000,BROOKLYN,K1,11214.0
4,2018-12-31T23:57:42.000,UNC,2,UNC,2,2019-01-01T00:00:39.000,2019-01-01T00:15:00.000,MANHATTAN,M3,10019.0
5,2018-12-31T23:57:41.000,SICMIN,7,SICMIN,7,2018-12-31T23:58:01.000,2019-01-01T00:25:32.000,BROOKLYN,K4,11208.0
6,2018-12-31T23:57:06.000,UNKNOW,4,UNKNOW,4,2018-12-31T23:57:17.000,2019-01-01T00:02:25.000,QUEENS,Q5,11435.0
7,2018-12-31T23:57:04.000,CARDBR,2,CARDBR,2,2018-12-31T23:57:16.000,2018-12-31T23:57:37.000,BRONX,B3,10465.0
8,2018-12-31T23:55:49.000,INJURY,5,INJURY,5,2018-12-31T23:56:20.000,2019-01-01T00:01:47.000,QUEENS,Q1,11691.0
9,2018-12-31T23:55:27.000,INJURY,5,INJURY,5,2018-12-31T23:56:02.000,2019-01-01T00:04:46.000,BRONX,B2,10453.0


In [11]:
d3 = d[['INCIDENT_DATETIME',
       'INITIAL_CALL_TYPE', 'INITIAL_SEVERITY_LEVEL_CODE', 'FINAL_CALL_TYPE',
       'FINAL_SEVERITY_LEVEL_CODE', 'FIRST_ASSIGNMENT_DATETIME', 'DISPATCH_RESPONSE_SECONDS_QY',
       'FIRST_ACTIVATION_DATETIME', 'FIRST_ON_SCENE_DATETIME', 'INCIDENT_RESPONSE_SECONDS_QY',
       'INCIDENT_TRAVEL_TM_SECONDS_QY', 'FIRST_TO_HOSP_DATETIME',
       'FIRST_HOSP_ARRIVAL_DATETIME', 'INCIDENT_CLOSE_DATETIME', 'BOROUGH',
       'INCIDENT_DISPATCH_AREA', 'ZIPCODE']]

In [12]:
d3

,INCIDENT_DATETIME,INITIAL_CALL_TYPE,INITIAL_SEVERITY_LEVEL_CODE,FINAL_CALL_TYPE,FINAL_SEVERITY_LEVEL_CODE,FIRST_ASSIGNMENT_DATETIME,DISPATCH_RESPONSE_SECONDS_QY,FIRST_ACTIVATION_DATETIME,FIRST_ON_SCENE_DATETIME,INCIDENT_RESPONSE_SECONDS_QY,INCIDENT_TRAVEL_TM_SECONDS_QY,FIRST_TO_HOSP_DATETIME,FIRST_HOSP_ARRIVAL_DATETIME,INCIDENT_CLOSE_DATETIME,BOROUGH,INCIDENT_DISPATCH_AREA,ZIPCODE
0,2018-12-31T23:59:46.000,CARDBR,2,CARDBR,2,2018-12-31T23:59:54.000,8,2019-01-01T00:00:51.000,2019-01-01T00:07:27.000,461.0,453.0,2019-01-01T00:15:08.000,2019-01-01T00:21:51.000,2019-01-01T01:00:20.000,BROOKLYN,K6,11201.0
1,2018-12-31T23:59:20.000,MVAINJ,4,MVAINJ,4,2019-01-01T00:01:10.000,110,2019-01-01T00:02:25.000,2019-01-01T00:07:19.000,479.0,369.0,NaN,NaN,2019-01-01T00:19:49.000,QUEENS,Q3,11416.0
2,2018-12-31T23:58:45.000,SICK,6,SICK,6,2019-01-01T00:00:43.000,118,2019-01-01T00:02:08.000,2019-01-01T00:14:18.000,933.0,815.0,2019-01-01T00:23:59.000,2019-01-01T00:34:07.000,2019-01-01T01:21:30.000,QUEENS,Q6,11369.0
3,2018-12-31T23:58:14.000,INJMAJ,3,INJMAJ,3,2018-12-31T23:58:22.000,8,2018-12-31T23:58:37.000,2019-01-01T00:04:10.000,356.0,348.0,2019-01-01T01:06:29.000,2019-01-01T01:15:30.000,2019-01-01T02:02:12.000,BROOKLYN,K1,11214.0
4,2018-12-31T23:57:42.000,UNC,2,UNC,2,2019-01-01T00:00:39.000,177,2019-01-01T00:01:03.000,2019-01-01T00:15:00.000,1038.0,861.0,2019-01-01T01:18:22.000,2019-01-01T01:22:01.000,2019-01-01T02:34:19.000,MANHATTAN,M3,10019.0
5,2018-12-31T23:57:41.000,SICMIN,7,SICMIN,7,2018-12-31T23:58:01.000,20,2018-12-31T23:58:14.000,2019-01-01T00:25:32.000,1671.0,1651.0,NaN,NaN,2019-01-01T00:55:18.000,BROOKLYN,K4,11208.0
6,2018-12-31T23:57:06.000,UNKNOW,4,UNKNOW,4,2018-12-31T23:57:17.000,11,2018-12-31T23:57:28.000,2019-01-01T00:02:25.000,319.0,308.0,2019-01-01T00:16:49.000,2019-01-01T00:20:39.000,2019-01-01T00:52:00.000,QUEENS,Q5,11435.0
7,2018-12-31T23:57:04.000,CARDBR,2,CARDBR,2,2018-12-31T23:57:16.000,12,2018-12-31T23:58:13.000,2018-12-31T23:57:37.000,33.0,21.0,2019-01-01T00:27:33.000,2019-01-01T00:44:22.000,2019-01-01T01:04:48.000,BRONX,B3,10465.0
8,2018-12-31T23:55:49.000,INJURY,5,INJURY,5,2018-12-31T23:56:20.000,31,2018-12-31T23:56:36.000,2019-01-01T00:01:47.000,358.0,327.0,2019-01-01T00:11:20.000,2019-01-01T00:14:21.000,2019-01-01T00:50:20.000,QUEENS,Q1,11691.0
9,2018-12-31T23:55:27.000,INJURY,5,INJURY,5,2018-12-31T23:56:02.000,35,2018-12-31T23:56:08.000,2019-01-01T00:04:46.000,559.0,524.0,2019-01-01T00:14:54.000,2019-01-01T00:32:01.000,2019-01-01T01:07:55.000,BRONX,B2,10453.0


## We used the above data to create a mapping of call types to severities

In [17]:
# help with severity map code

# dic = {'CARDBR': {1: 1}}
# 2 in dic['CARDBR']
# dic['CARDBR'][1] += 1
# dic['CARDBR'][2] = 1
# dic

{'CARDBR': {1: 2, 2: 1}}

In [18]:
########################
####   HERE WE GO   ####
########################

severity_map = {}
for index, row in d.iterrows():
    if row['INITIAL_CALL_TYPE'] not in severity_map.keys():
        severity_map[row['INITIAL_CALL_TYPE']] = {row['INITIAL_SEVERITY_LEVEL_CODE']: 1}
    else:
        if row['INITIAL_SEVERITY_LEVEL_CODE'] not in severity_map[row['INITIAL_CALL_TYPE']]:
            severity_map[row['INITIAL_CALL_TYPE']][row['INITIAL_SEVERITY_LEVEL_CODE']] = 1
        else:
            severity_map[row['INITIAL_CALL_TYPE']][row['INITIAL_SEVERITY_LEVEL_CODE']] += 1
            
severity_map

{'CARDBR': {2: 70590, 5: 2, 4: 3, 7: 2},
 'MVAINJ': {4: 46555, 3: 38, 2: 57, 7: 1, 1: 3, 5: 8, 6: 9},
 'SICK': {6: 226500, 7: 28, 4: 325, 2: 27, 3: 40, 5: 22, 1: 5},
 'INJMAJ': {3: 46574, 4: 9, 1: 1, 7: 1, 5: 10, 2: 7, 6: 3},
 'UNC': {2: 79029, 1: 13, 6: 10, 3: 14, 4: 35, 5: 8},
 'SICMIN': {7: 15799, 4: 9, 3: 3, 6: 23, 5: 3, 1: 1, 2: 1},
 'UNKNOW': {4: 118878, 5: 37, 2: 63, 6: 42, 1: 13, 8: 1, 3: 76, 7: 17},
 'INJURY': {5: 181364, 4: 3166, 2: 57, 7: 35, 1: 3, 6: 18, 3: 97},
 'ASTHMB': {2: 12206, 3: 808},
 'DRUG': {4: 101580, 5: 14, 1: 3, 2: 35, 7: 28, 3: 19, 6: 2},
 'STATEP': {2: 25807, 4: 9, 6: 3, 3: 15, 1: 1},
 'DIFFBR': {2: 95893, 3: 9, 5: 3, 1: 10, 6: 5, 7: 1, 4: 6},
 'SICPED': {4: 9429, 3: 2, 2: 6, 5: 2, 1: 1},
 'SEIZR': {3: 12944, 2: 6, 4: 1},
 'EDP': {7: 139885, 4: 108, 6: 31, 3: 38, 5: 81, 1: 5, 2: 24, 8: 1},
 'PEDSTR': {3: 22518, 2: 15, 5: 8, 4: 8, 1: 1},
 'INBLED': {3: 11639, 1: 1},
 'CARD': {3: 54753, 1: 1, 2: 14, 6: 2, 4: 6},
 'TRAUMA': {2: 5851, 4: 24, 3: 6, 5: 1, 8: 1, 7:

In [15]:
severity_map2 = {'CARDBR': [2, 5, 4, 7],
 'MVAINJ': [4, 3, 2, 7, 1, 5, 6],
 'SICK': [6, 7, 4, 2, 3, 5, 1],
 'INJMAJ': [3, 4, 1, 7, 5, 2, 6],
 'UNC': [2, 1, 6, 3, 4, 5],
 'SICMIN': [7, 4, 3, 6, 5, 1, 2],
 'UNKNOW': [4, 5, 2, 6, 1, 8, 3, 7],
 'INJURY': [5, 4, 2, 7, 1, 6, 3],
 'ASTHMB': [2, 3],
 'DRUG': [4, 5, 1, 2, 7, 3, 6],
 'STATEP': [2, 4, 6, 3, 1],
 'DIFFBR': [2, 3, 5, 1, 6, 7, 4],
 'SICPED': [4, 3, 2, 5, 1],
 'SEIZR': [3, 2, 4],
 'EDP': [7, 4, 6, 3, 5, 1, 2, 8],
 'PEDSTR': [3, 2, 5, 4, 1],
 'INBLED': [3, 1],
 'CARD': [3, 1, 2, 6, 4],
 'TRAUMA': [2, 4, 3, 5, 8, 7, 6, 1],
 'INJMIN': [7, 4, 3, 5, 6],
 'ABDPN': [5, 4, 3, 2, 7],
 'ANAPH': [2, 1, 4, 6],
 'GYNMAJ': [3],
 'OBLAB': [5, 4],
 'CVAC': [2, 4, 7, 3],
 'EDPC': [5, 3, 7, 4],
 'STAB': [3, 1, 5, 4, 2],
 'JUMPUP': [7, 4, 5, 2],
 'PD13': [7, 3, 5],
 'ALTMEN': [3, 6, 5, 1, 7, 2, 4],
 'OTHER': [6, 7, 4, 3, 2, 5],
 'RESPIR': [4, 2, 1, 7, 3],
 'ARREST': [1, 7, 4, 2, 3],
 'MEDRXN': [5, 3, 4, 2],
 'HYPTN': [7, 4, 6, 2, 3],
 'CHOKE': [1, 4],
 'MVA': [6, 4],
 'SHOT': [3, 2, 1],
 'OBMIS': [4, 3],
 'GYNHEM': [5, 4, 3, 1],
 'SAFE': [6, 7, 4, 5],
 'INHALE': [5, 2, 4, 6, 1, 3],
 'CVA': [4, 3],
 'BURNMA': [3, 2],
 'COLD': [5, 2],
 'STNDBY': [8, 5, 7, 4, 2, 6, 3, 1],
 'STRANS': [8, 2],
 'OBMAJ': [3, 5, 2],
 'BURNMI': [4, 7, 1, 3],
 'JUMPDN': [2, 5, 7],
 'OBCOMP': [2, 4],
 'MCI21': [2, 4, 5, 8],
 'OBOUT': [3],
 'DROWN': [2, 6, 1],
 'EDPW': [7, 5],
 'AMPMIN': [5, 3, 4],
 'ACTIVE': [2],
 'CHILDA': [6, 4],
 'SPEVNT': [8],
 'ELECT': [3],
 'MCI77': [2, 1],
 'VENOM': [2],
 'MEDVAC': [2],
 'MVAINM': [2],
 'HEAT': [4],
 'DIFFFC': [2],
 'SICKFC': [6, 3, 4, 2, 1],
 'MCI22': [2],
 'MCI76': [2],
 'MOSILL': [9],
 'AMPMAJ': [3],
 'PD13C': [3],
 'TRAUMS': [2],
 'BBP': [9],
 'MCI29': [2],
 'MOSINJ': [9, 2],
 'DOA': [7],
 'PEDFC': [4],
 'ACC': [9],
 'CARDFC': [3],
 'ABDPFC': [5],
 'RESPFC': [4, 3, 5, 2],
 'ASTHFC': [2, 3],
 'NOVEH': [9],
 'STATFC': [2],
 'MEDRFC': [4, 5],
 'UNCFC': [2],
 'FIRE77': [4],
 'ADM': [9],
 'PEDRF': [4],
 'MCI21P': [2]}

#CONCLUSION: The same call type does not always map the same severity #

### We calculated the most common classification of call type to severity to use for our project 

In [25]:
sev_modes = {}
for i in severity_map.keys():
    maxOcc = 0
    for j in severity_map[i].keys():
        if severity_map[i][j] > maxOcc:
            mode = j
    sev_modes[i] = mode
    
sev_modes

{'CARDBR': 7,
 'MVAINJ': 6,
 'SICK': 1,
 'INJMAJ': 6,
 'UNC': 5,
 'SICMIN': 2,
 'UNKNOW': 7,
 'INJURY': 3,
 'ASTHMB': 3,
 'DRUG': 6,
 'STATEP': 1,
 'DIFFBR': 4,
 'SICPED': 1,
 'SEIZR': 4,
 'EDP': 8,
 'PEDSTR': 1,
 'INBLED': 1,
 'CARD': 4,
 'TRAUMA': 1,
 'INJMIN': 6,
 'ABDPN': 7,
 'ANAPH': 6,
 'GYNMAJ': 3,
 'OBLAB': 4,
 'CVAC': 3,
 'EDPC': 4,
 'STAB': 2,
 'JUMPUP': 2,
 'PD13': 5,
 'ALTMEN': 4,
 'OTHER': 5,
 'RESPIR': 3,
 'ARREST': 3,
 'MEDRXN': 2,
 'HYPTN': 3,
 'CHOKE': 4,
 'MVA': 4,
 'SHOT': 1,
 'OBMIS': 3,
 'GYNHEM': 1,
 'SAFE': 5,
 'INHALE': 3,
 'CVA': 3,
 'BURNMA': 2,
 'COLD': 2,
 'STNDBY': 1,
 'STRANS': 2,
 'OBMAJ': 2,
 'BURNMI': 3,
 'JUMPDN': 7,
 'OBCOMP': 4,
 'MCI21': 8,
 'OBOUT': 3,
 'DROWN': 1,
 'EDPW': 5,
 'AMPMIN': 4,
 'ACTIVE': 2,
 'CHILDA': 4,
 'SPEVNT': 8,
 'ELECT': 3,
 'MCI77': 1,
 'VENOM': 2,
 'MEDVAC': 2,
 'MVAINM': 2,
 'HEAT': 4,
 'DIFFFC': 2,
 'SICKFC': 1,
 'MCI22': 2,
 'MCI76': 2,
 'MOSILL': 9,
 'AMPMAJ': 3,
 'PD13C': 3,
 'TRAUMS': 2,
 'BBP': 9,
 'MCI29': 2,
 'MOSINJ

## At this point, we have attained the information on the Bronx, from 2015-2018

In [4]:
br = pd.read_csv("bronx.csv")
br

,Unnamed: 0,CAD_INCIDENT_ID,INCIDENT_DATETIME,INITIAL_CALL_TYPE,INITIAL_SEVERITY_LEVEL_CODE,FINAL_CALL_TYPE,FINAL_SEVERITY_LEVEL_CODE,FIRST_ASSIGNMENT_DATETIME,VALID_DISPATCH_RSPNS_TIME_INDC,DISPATCH_RESPONSE_SECONDS_QY,...,ZIPCODE,POLICEPRECINCT,CITYCOUNCILDISTRICT,COMMUNITYDISTRICT,COMMUNITYSCHOOLDISTRICT,CONGRESSIONALDISTRICT,REOPEN_INDICATOR,SPECIAL_EVENT_INDICATOR,STANDBY_INDICATOR,TRANSFER_INDICATOR
0,7,183654379,2018-12-31T23:57:04.000,CARDBR,2,CARDBR,2,2018-12-31T23:57:16.000,Y,12,...,10465.0,45.0,13.0,210.0,8.0,14.0,N,N,N,N
1,9,183654376,2018-12-31T23:55:27.000,INJURY,5,INJURY,5,2018-12-31T23:56:02.000,Y,35,...,10453.0,46.0,14.0,205.0,9.0,15.0,N,N,N,N
2,18,183654365,2018-12-31T23:51:39.000,SEIZR,3,SEIZR,3,2018-12-31T23:52:12.000,Y,33,...,10454.0,40.0,17.0,201.0,7.0,15.0,N,N,N,N
3,20,183654363,2018-12-31T23:51:00.000,MVAINJ,4,MVAINJ,4,2018-12-31T23:54:00.000,Y,180,...,10459.0,41.0,17.0,202.0,8.0,15.0,N,N,N,N
4,23,183654360,2018-12-31T23:50:38.000,EDP,7,EDPC,5,2018-12-31T23:58:53.000,Y,495,...,10467.0,49.0,15.0,211.0,11.0,14.0,N,N,N,N
5,24,183654359,2018-12-31T23:50:12.000,PEDSTR,3,PEDSTR,3,2018-12-31T23:51:48.000,Y,96,...,10456.0,44.0,16.0,204.0,9.0,15.0,N,N,N,N
6,34,183654344,2018-12-31T23:47:19.000,UNKNOW,4,UNKNOW,4,2018-12-31T23:58:36.000,Y,677,...,10469.0,49.0,13.0,211.0,11.0,14.0,N,N,N,N
7,44,183654331,2018-12-31T23:45:29.000,DRUG,4,DRUG,4,2018-12-31T23:48:50.000,Y,201,...,10462.0,43.0,18.0,209.0,11.0,14.0,N,N,N,N
8,47,183654326,2018-12-31T23:44:03.000,TRAUMA,2,TRAUMA,2,2018-12-31T23:44:39.000,Y,36,...,10456.0,42.0,16.0,203.0,9.0,15.0,N,N,N,N
9,54,183654318,2018-12-31T23:42:35.000,SICK,6,SICK,6,2018-12-31T23:43:07.000,Y,32,...,10461.0,49.0,13.0,211.0,11.0,14.0,N,N,N,N


#### We now calculate the severity mappings for just the Bronx, because that is the data we will be using

In [5]:
severity_map_br = {}
for index, row in br.iterrows():
    if row['INITIAL_CALL_TYPE'] not in severity_map_br.keys():
        severity_map_br[row['INITIAL_CALL_TYPE']] = {row['INITIAL_SEVERITY_LEVEL_CODE']: 1}
    else:
        if row['INITIAL_SEVERITY_LEVEL_CODE'] not in severity_map_br[row['INITIAL_CALL_TYPE']]:
            severity_map_br[row['INITIAL_CALL_TYPE']][row['INITIAL_SEVERITY_LEVEL_CODE']] = 1
        else:
            severity_map_br[row['INITIAL_CALL_TYPE']][row['INITIAL_SEVERITY_LEVEL_CODE']] += 1
            
sev_modes_br = {}
for i in severity_map_br.keys():
    maxOcc = 0
    for j in severity_map_br[i].keys():
        if severity_map_br[i][j] > maxOcc:
            mode = j
    sev_modes_br[i] = mode
    
sev_modes_br

{'CARDBR': 3,
 'INJURY': 8,
 'SEIZR': 4,
 'MVAINJ': 6,
 'EDP': 8,
 'PEDSTR': 7,
 'UNKNOW': 1,
 'DRUG': 1,
 'TRAUMA': 1,
 'SICK': 1,
 'SICPED': 5,
 'OBLAB': 2,
 'UNC': 4,
 'INJMIN': 1,
 'INBLED': 2,
 'INJMAJ': 6,
 'DIFFBR': 7,
 'CARD': 2,
 'ASTHMB': 1,
 'SICMIN': 3,
 'EDPC': 4,
 'OTHER': 5,
 'STATEP': 1,
 'SHOT': 1,
 'ALTMEN': 2,
 'ARREST': 6,
 'GYNHEM': 2,
 'SAFE': 4,
 'CVAC': 3,
 'ABDPN': 1,
 'INHALE': 3,
 'CVA': 3,
 'RESPIR': 6,
 'STAB': 2,
 'HYPTN': 4,
 'MEDRXN': 4,
 'COLD': 4,
 'ANAPH': 4,
 'OBMAJ': 2,
 'BURNMI': 5,
 'JUMPDN': 1,
 'CHOKE': 4,
 'PD13': 6,
 'OBMIS': 4,
 'GYNMAJ': 5,
 'STNDBY': 6,
 'OBOUT': 3,
 'BURNMA': 7,
 'ACTIVE': 3,
 'MCI21': 8,
 'OBCOMP': 3,
 'STRANS': 2,
 'AMPMIN': 4,
 'CHILDA': 5,
 'MVA': 2,
 'DROWN': 2,
 'HEAT': 3,
 'ELECT': 3,
 'JUMPUP': 5,
 'SICKFC': 5,
 'MCI77': 1,
 'PD13C': 3,
 'VENOM': 2,
 'AMPMAJ': 3,
 'BBP': 7,
 'DIFFFC': 2,
 'DOA': 7,
 'PEDFC': 4,
 'MCI76': 2,
 'SPEVNT': 8,
 'MEDVAC': 2,
 'ACC': 9,
 'CARDFC': 3,
 'ABDPFC': 4,
 'RESPFC': 4,
 'ASTHFC': 

### started working on call frequency calculations

In [7]:
br['INCIDENT_DATETIME'][0]

'2018-12-31T23:59:46.000'

In [11]:
br['INCIDENT_DATETIME'][0][11:]

'23:59:46.000'

In [12]:
import datetime as dt

In [24]:
idt = br['INCIDENT_DATETIME'][0]
idt = idt.replace("T", " ")
print(idt)
dt.datetime.strptime(idt, '%Y-%m-%d %H:%M:%S.%f')

2018-12-31 23:59:46.000


datetime.datetime(2018, 12, 31, 23, 59, 46)

In [6]:
datetimes = []
for row in br['INCIDENT_DATETIME']:
    idt = row.replace("T", " ")
    datetimes.append(dt.datetime.strptime(idt, '%Y-%m-%d %H:%M:%S.%f'))
    
datetimes

NameError: name 'dt' is not defined

In [11]:
type(br.loc[0]['FIRST_ON_SCENE_DATETIME']) == 'str'

False